## Part 1:  PHPMyAdmin

QUESTIONS:
* Q1: Some of the facilities charge a fee to members, but some do not. Write a SQL query to produce a list of the names of the facilities that do.

`SELECT name FROM Facilities WHERE membercost <> 0;`

name	
Tennis Court 1	
Tennis Court 2	
Massage Room 1	
Massage Room 2	
Squash Court


* Q2: How many facilities do not charge a fee to members?

`SELECT COUNT(name) FROM Facilities WHERE membercost = 0;`

4	


* Q3: Write an SQL query to show a list of facilities that charge a fee to members, where the fee is less than 20% of the facility's monthly maintenance cost. Return the facid, facility name, member cost, and monthly maintenance of the facilities in question.

`SELECT facid, name, membercost, monthlymaintenance FROM Facilities WHERE membercost <> 0 AND membercost < (monthlymaintenance/5);`

facid	name	membercost	monthlymaintenance	
0	Tennis Court 1	5.0	200	
1	Tennis Court 2	5.0	200	
4	Massage Room 1	9.9	3000	
5	Massage Room 2	9.9	3000	
6	Squash Court	3.5	80	


* Q4: Write an SQL query to retrieve the details of facilities with ID 1 and 5. Try writing the query without using the OR operator.

`SELECT * FROM Facilities WHERE facid IN (1,5);`

facid	name	membercost	guestcost	initialoutlay	monthlymaintenance	
1	Tennis Court 2	5.0	25.0	8000	200	
5	Massage Room 2	9.9	80.0	4000	3000	


* Q5: Produce a list of facilities, with each labelled as 'cheap' or 'expensive', depending on if their monthly maintenance cost is more than $100. Return the name and monthly maintenance of the facilities in question.

`SELECT name, monthlymaintenance, 
CASE WHEN monthlymaintenance > 100 THEN 'expensive'
ELSE 'cheap' END
AS price_category
FROM Facilities;`

name	monthlymaintenance	price_category	
Tennis Court 1	200	expensive	
Tennis Court 2	200	expensive	
Badminton Court	50	cheap	
Table Tennis	10	cheap	
Massage Room 1	3000	expensive	
Massage Room 2	3000	expensive	
Squash Court	80	cheap	
Snooker Table	15	cheap	
Pool Table	15	cheap	

* Q6: You'd like to get the first and last name of the last member(s) who signed up. Try not to use the LIMIT clause for your solution.

`SELECT firstname, surname FROM Members WHERE joindate >= '2012-09-15' ORDER BY joindate DESC;`

firstname	surname	
Darren	Smith	
Erica	Crumpet	
John	Hunt	
Hyacinth	Tupperware	
Millicent	Purview	
Henry	Worthington-Smyth	
David	Farrell	


* Q7: Produce a list of all members who have used a tennis court.
Include in your output the name of the court, and the name of the member
formatted as a single column. Ensure no duplicate data, and order by
the member name.

`SELECT DISTINCT CONCAT(firstname, " ", surname) AS full_member_name, name as facility_name
FROM Members 
INNER JOIN Bookings ON Members.memid = Bookings.memid 
INNER JOIN Facilities ON Facilities.facid = Bookings.facid
WHERE name LIKE 'Tennis%'
ORDER BY full_member_name;`

full_member_name   	facility_name	
Anne Baker	Tennis Court 1	
Anne Baker	Tennis Court 2	
Burton Tracy	Tennis Court 1	
Burton Tracy	Tennis Court 2	
Charles Owen	Tennis Court 2	
Charles Owen	Tennis Court 1	
Darren Smith	Tennis Court 2	
David Farrell	Tennis Court 1	
David Farrell	Tennis Court 2	
David Jones	Tennis Court 2	
David Jones	Tennis Court 1	
David Pinker	Tennis Court 1	
Douglas Jones	Tennis Court 1	
Erica Crumpet	Tennis Court 1	
Florence Bader	Tennis Court 1	
Florence Bader	Tennis Court 2	
Gerald Butters	Tennis Court 1	
Gerald Butters	Tennis Court 2	
GUEST GUEST	Tennis Court 2	
GUEST GUEST	Tennis Court 1	
Henrietta Rumney	Tennis Court 2	
Jack Smith	Tennis Court 2	
Jack Smith	Tennis Court 1	
Janice Joplette	Tennis Court 2	
Janice Joplette	Tennis Court 1	


* Q8: Produce a list of bookings on the day of 2012-09-14 which
will cost the member (or guest) more than $30. Remember that guests have
different costs to members (the listed costs are per half-hour 'slot'), and
the guest user's ID is always 0. Include in your output the name of the
facility, the name of the member formatted as a single column, and the cost.
Order by descending cost, and do not use any subqueries.

`SELECT Facilities.name as facility, CONCAT(firstname, " ", surname) AS member,
CASE WHEN (Members.memid = 0) THEN guestcost * slots
ELSE membercost * slots END
AS cost
FROM Bookings
INNER JOIN Facilities
ON Bookings.facid = Facilities.facid
INNER JOIN Members
ON Bookings.memid = Members.memid
WHERE starttime LIKE '2012-09-14%' AND ((Members.memid <> 0 AND membercost*slots > 30) OR (Members.memid = 0 AND guestcost*slots > 30))
ORDER BY cost DESC;`

facility	member	cost   	
Massage Room 2	GUEST GUEST	320.0	
Massage Room 1	GUEST GUEST	160.0	
Massage Room 1	GUEST GUEST	160.0	
Massage Room 1	GUEST GUEST	160.0	
Tennis Court 2	GUEST GUEST	150.0	
Tennis Court 1	GUEST GUEST	75.0	
Tennis Court 2	GUEST GUEST	75.0	
Tennis Court 1	GUEST GUEST	75.0	
Squash Court	GUEST GUEST	70.0	
Massage Room 1	Jemima Farrell	39.6	
Squash Court	GUEST GUEST	35.0	
Squash Court	GUEST GUEST	35.0	


* Q9: This time, produce the same result as in Q8, but using a subquery.

`SELECT Facilities.name as facility, CONCAT(firstname, " ", surname) AS member,
CASE WHEN (Members.memid = 0) THEN guestcost * slots
ELSE membercost * slots END
AS cost
FROM Bookings
INNER JOIN Facilities
ON Bookings.facid = Facilities.facid
INNER JOIN Members
ON Bookings.memid = Members.memid
WHERE bookid IN (SELECT bookid FROM Bookings WHERE starttime LIKE '2012-09-14%') AND ((Members.memid <> 0 AND membercost*slots > 30) OR (Members.memid = 0 AND guestcost*slots > 30))
ORDER BY cost DESC;`


## Part 2: SQLite


In [97]:
# Code copied from LocalSQLConnection.py:
import sqlite3
from sqlite3 import Error

 
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        #print(sqlite3.version)
    except Error as e:
        print(e)
 
    return conn

 
def execute_query(conn, query_string):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    
#     query1 = """
#         SELECT *
#         FROM FACILITIES
#         """
#   cur.execute(query1)

    cur.execute(query_string)
 
    rows = cur.fetchall()
     
    for row in rows:
        print(row)
        
def query(query_string):
    database = "sqlite_db_pythonsqlite.db"
 
    # create a database connection
    conn = create_connection(database)
    with conn: 
        execute_query(conn, query_string)


QUESTIONS:
* Q10: Produce a list of facilities with a total revenue less than 1000. The output of facility name and total revenue, sorted by revenue. Remember that there's a different cost for guests and members!

In [98]:
# "WHERE revenue < 1000" between USING and GROUP doesn't work (can't use alias 'revenue' in WHERE clause)
# solution is to use subquery as below 
# (from https://stackoverflow.com/questions/69680151/sql-filter-based-on-case-statement)
# result is empty if using < 1000; wrong SQL?

Q10 = """
    SELECT * FROM 
    (
        SELECT name, 
        CASE WHEN (memid = 0) THEN  SUM(slots * guestcost)
        ELSE  SUM(slots * membercost) END
        AS revenue
        FROM Facilities
        LEFT JOIN Bookings
        USING(facid)
        GROUP BY name
        ORDER BY revenue
        ) x
    WHERE x.revenue < 10000;
    """
query(Q10)

('Table Tennis', 4150)
('Snooker Table', 4540)
('Pool Table', 4550)


* Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order

In [103]:
Q11 = """
    SELECT m1.surname, m1.firstname, m2.surname, m2.firstname
    FROM Members as m1
    INNER JOIN Members as m2
    ON m1.recommendedby = m2.memid
    ORDER BY m2.surname, m2.firstname;
    """
query(Q11)

('Sarwin', 'Ramnaresh', 'Bader', 'Florence')
('Coplin', 'Joan', 'Baker', 'Timothy')
('Genting', 'Matthew', 'Butters', 'Gerald')
('Baker', 'Timothy', 'Farrell', 'Jemima')
('Pinker', 'David', 'Farrell', 'Jemima')
('Rumney', 'Henrietta', 'Genting', 'Matthew')
('Jones', 'Douglas', 'Jones', 'David')
('Dare', 'Nancy', 'Joplette', 'Janice')
('Jones', 'David', 'Joplette', 'Janice')
('Hunt', 'John', 'Purview', 'Millicent')
('Boothe', 'Tim', 'Rownam', 'Tim')
('Joplette', 'Janice', 'Smith', 'Darren')
('Butters', 'Gerald', 'Smith', 'Darren')
('Owen', 'Charles', 'Smith', 'Darren')
('Smith', 'Jack', 'Smith', 'Darren')
('Mackenzie', 'Anna', 'Smith', 'Darren')
('Worthington-Smyth', 'Henry', 'Smith', 'Tracy')
('Purview', 'Millicent', 'Smith', 'Tracy')
('Crumpet', 'Erica', 'Smith', 'Tracy')
('Baker', 'Anne', 'Stibbons', 'Ponder')
('Bader', 'Florence', 'Stibbons', 'Ponder')
('Stibbons', 'Ponder', 'Tracy', 'Burton')


* Q12: Find the facilities with their usage by member, but not guests

In [104]:
Q12 = """
    SELECT name, m.surname, m.firstname, COUNT(*) AS count
    FROM Facilities 
    LEFT JOIN Bookings as b USING (facid)
    LEFT JOIN Members as m USING (memid)
    WHERE b.memid <> 0
    GROUP BY name, m.memid
    ORDER BY name, m.surname, m.firstname;
    """
query(Q12)

('Badminton Court', 'Bader', 'Florence', 9)
('Badminton Court', 'Baker', 'Anne', 10)
('Badminton Court', 'Baker', 'Timothy', 7)
('Badminton Court', 'Boothe', 'Tim', 12)
('Badminton Court', 'Butters', 'Gerald', 20)
('Badminton Court', 'Crumpet', 'Erica', 2)
('Badminton Court', 'Dare', 'Nancy', 10)
('Badminton Court', 'Farrell', 'Jemima', 7)
('Badminton Court', 'Hunt', 'John', 2)
('Badminton Court', 'Jones', 'David', 8)
('Badminton Court', 'Jones', 'Douglas', 2)
('Badminton Court', 'Mackenzie', 'Anna', 30)
('Badminton Court', 'Owen', 'Charles', 6)
('Badminton Court', 'Pinker', 'David', 7)
('Badminton Court', 'Purview', 'Millicent', 2)
('Badminton Court', 'Rownam', 'Tim', 4)
('Badminton Court', 'Sarwin', 'Ramnaresh', 7)
('Badminton Court', 'Smith', 'Darren', 132)
('Badminton Court', 'Smith', 'Jack', 12)
('Badminton Court', 'Smith', 'Tracy', 32)
('Badminton Court', 'Stibbons', 'Ponder', 16)
('Badminton Court', 'Tracy', 'Burton', 2)
('Badminton Court', 'Tupperware', 'Hyacinth', 1)
('Badmint

* Q13: Find the facilities usage by month, but not guests

In [105]:
Q13 = """
    SELECT name as facility, STRFTIME('%Y-%m', starttime) AS month, COUNT(name) AS count
    FROM Bookings
    LEFT JOIN Facilities USING (facid)
    WHERE memid <> 0
    GROUP BY name, month
    ORDER BY name, month
    """
query(Q13)

('Badminton Court', '2012-07', 51)
('Badminton Court', '2012-08', 132)
('Badminton Court', '2012-09', 161)
('Massage Room 1', '2012-07', 77)
('Massage Room 1', '2012-08', 153)
('Massage Room 1', '2012-09', 191)
('Massage Room 2', '2012-07', 4)
('Massage Room 2', '2012-08', 9)
('Massage Room 2', '2012-09', 14)
('Pool Table', '2012-07', 103)
('Pool Table', '2012-08', 272)
('Pool Table', '2012-09', 408)
('Snooker Table', '2012-07', 68)
('Snooker Table', '2012-08', 154)
('Snooker Table', '2012-09', 199)
('Squash Court', '2012-07', 23)
('Squash Court', '2012-08', 85)
('Squash Court', '2012-09', 87)
('Table Tennis', '2012-07', 48)
('Table Tennis', '2012-08', 143)
('Table Tennis', '2012-09', 194)
('Tennis Court 1', '2012-07', 65)
('Tennis Court 1', '2012-08', 111)
('Tennis Court 1', '2012-09', 132)
('Tennis Court 2', '2012-07', 41)
('Tennis Court 2', '2012-08', 109)
('Tennis Court 2', '2012-09', 126)
